# Primeiro time a marcar um gol

Este notebook tem como principal objetivo utilizar o algorito K-Means para determinar qual time possui a maior probabilidade de marcar o primeiro gol em uma partida.

### Random Forest e K-Means - Algoritmos utilizado no modelo

O Random Forest é um algoritmo de aprendizado supervisionado que usa uma coleção de várias árvores de decisão (floresta) para fazer previsões. Ele cria várias árvores de forma aleatória e combina seus resultados para melhorar a precisão e evitar o overfitting. Cada árvore contribui com um "voto" para a previsão final, sendo uma média ou uma votação majoritária. Já o K-Means é um algoritmo de aprendizado não supervisionado que agrupa dados em "K" clusters com base em suas características. Ele funciona atribuindo cada ponto de dado ao cluster mais próximo (com base na distância euclidiana), recalculando as médias dos clusters e repetindo até que as atribuições não mudem mais.

Ambos os algoritmos são utilizados neste notebook, uma vez que o Random Forest é utilizado para calcular as probabilidades de cada time fazer o primeiro gol e o K-Means é responsável por criar os clusters e separar os times em grupos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
matches = pd.read_csv('../../data/brazil-serie-a-matches-2024-to-2024-stats (5).csv')
times1 = pd.read_csv('../../data/brazil-serie-a-teams-2024-to-2024-stats (4).csv')
times2 = pd.read_csv('../../data/brazil-serie-a-teams2-2024-to-2024-stats (2).csv')

In [3]:

teams = pd.merge(times1, times2, on="team_name", how="left")

teams.insert(0, "team_index", list(range(1, 21)))

In [4]:
selected_tables = [
    "team_index",
    "wins",
    "draws",
    "losses",
    "first_team_to_score_count",
    "goals_scored",
    "goals_conceded",
    "goals_scored_half_time",
    "goals_conceded_half_time",
    "goal_difference_half_time",
    "clean_sheets",
    "btts_count",
    "fts_count",
    "leading_at_half_time",
    "draw_at_half_time",
    "losing_at_half_time",
    "scoredBothHalves_overall"
]

number_data = pd.DataFrame(teams[selected_tables])

selected_tables.insert(1, "team_name")

data = pd.DataFrame(teams[selected_tables])

---

In [5]:
x = number_data
y = number_data["team_index"]

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x, y)

In [7]:
probabilities = rf.predict_proba(number_data)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(probabilities)
centroids = kmeans.cluster_centers_

In [ ]:
silhouette_avg = silhouette_score(probabilities, clusters)
print("Silhouette coefficient:", silhouette_avg)

plt.scatter(data[clusters == 0]['wins'], data[clusters == 0]['losses'], color='red', label='Grandes')
plt.scatter(data[clusters == 1]['wins'], data[clusters == 1]['losses'], color='blue', label='Médio')
plt.scatter(data[clusters == 2]['wins'], data[clusters == 2]['losses'], color='green', label='Pequeno')
plt.scatter(centroids[:, 0], centroids[:, 1], color='black', marker='x', label='Centroids')
plt.xlabel('Wins')
plt.ylabel('Losses')
plt.title('Clusters of Teams')
plt.legend()
plt.show()

In [ ]:
probabilities = rf.predict_proba(number_data)

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(probabilities)

results_rf = pd.DataFrame()

results_rf['team_name'] = data["team_name"]
results_rf['prob_class_1'] = probabilities[:, 0]
results_rf['prob_class_2'] = probabilities[:, 1]
results_rf['prob_class_3'] = probabilities[:, 2]
results_rf['cluster'] = clusters

centroids = kmeans.cluster_centers_

plt.scatter(data[results_rf['cluster'] == 0]['wins'], data[results_rf['cluster'] == 0]['losses'], color='red', label='Grandes')
plt.scatter(data[results_rf['cluster'] == 1]['wins'], data[results_rf['cluster'] == 1]['losses'], color='blue', label='Médio')
plt.scatter(data[results_rf['cluster'] == 2]['wins'], data[results_rf['cluster'] == 2]['losses'], color='green', label='Pequeno')

plt.scatter(centroids[:, 0], centroids[:, 1], color='black', marker='x', label='Centroids')

plt.xlabel('Wins')
plt.ylabel('Losses')
plt.title('Clusters of Teams')
plt.legend()
plt.show()

In [ ]:
print(results_rf)